The Hessian of a circuit
-------------------------

Challenge Statement
You may already be familiar with the calculation of the gradient of a circuit, which can be done very easily thanks to the parameter-shift rule. But what about second derivatives? In calculus, the second derivative of a real function $\mathbb{R}^n \rightarrow \mathbb{R}$ is described via a matrix
$H_f$
called the Hessian:
$$H_f = \begin{pmatrix}
\frac{\partial^2 f}{\partial x_1^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2} & \cdots & \frac{\partial^2 f}{\partial x_1 \partial x_n} \\
\frac{\partial^2 f}{\partial x_2 \partial x_1} & \frac{\partial^2 f}{\partial x_2^2} & \cdots & \frac{\partial^2 f}{\partial x_2 \partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial^2 f}{\partial x_n \partial x_1} & \frac{\partial^2 f}{\partial x_n \partial x_2} & \cdots & \frac{\partial^2 f}{\partial x_n^2}
\end{pmatrix}$$

A variational quantum circuit usually depends on some parameters $\theta_1, ... , \theta_n$ and returns an expectation value, so it can also be seen as a function $\mathbb{R}^n \rightarrow \mathbb{R}$ Therefore, we can calculate its Hessian and, moreover, there's a parameter-shift rule that allows us to do it. Namely, the elements of the Hessian matrix for such circuit can be calculated via
$\frac{\partial^2 f}{\partial \theta_i \partial \theta_j}(\boldsymbol{\theta}) = \frac{\left[ f(\boldsymbol{\theta} + s(\hat{e}_i + \hat{e}_j)) - f(\boldsymbol{\theta} + s(-\hat{e}_i + \hat{e}_j)) - f(\boldsymbol{\theta} + s(\hat{e}_i - \hat{e}_j)) + f(\boldsymbol{\theta} - s(\hat{e}_i + \hat{e}_j)) \right]}{4 \sin^2(s)}$

where s can be any real value, and $\hat{e}_i$ is the i-th coordinate basis vector.

PennyLane has the built-in function qml.gradients.param_shift_hessian that uses this formula to compute the Hessian. In this challenge we will use it to calculate the Hessian of circuits with an arbitrary number of wires that are of the form
Note that, in this case, the number of wires and the number of parameters are related:
![circuit](./images/TheHessianofacircuit_1.png)
number of parameters = number of wires+2.

Challenge code
-------------------

You must complete the `compute_hessian` function, which should

Define a QNode which implements the circuit above and returns the expectation value $Z_0 \otimes Z_{n-1}$, where n is the number of wires `num_wires` (`int`).
Return the Hessian of the circuit evaluated on some parameters, enconded in the arguments `w` (`np.ndarray`). These parameters are differentiable. The Hessian should be encoded in either a tuple, which is the built-in return type for qml.gradients.param_shift_hessian, or an np.ndarray of shape (5,5) if you decide to compute the Hessian in a different way.

Input
-----------

As input to this problem, you are given a list of the form [num_wires, w], where

* `num_wires` (int) is the number of wires in the circuit.
* `w` is the a list of length num_wires + 2, which contains the values of the parameters at which the Hessian should be evaluated.

Output
------------

The output of your code should be the Hessian. Regardless of whether it's a tuple or an np.ndarray, it will be converted into a numpy array by the evaluation functions, which is what you'll see in the expected_output.

In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

def compute_hessian(num_wires, w):
    """
    This function must create a circuit with num_wire qubits
    as per the challenge statement and return the Hessian of such
    circuit evaluated on w.

    Args:
        - num_wires = The number of wires in the circuit
        - w (np.ndarray): A list of length num_wires + 2 containing float parameters.
        The Hessian is evaluated at this point in parameter space.

    Returns:
        Union(tuple, np.ndarray): A matrix representing the Hessian calculated via
        qml.gradients.parameter_shift_hessian
    """



    # Define your device and QNode

    # Return the Hessian using qml.gradient.param_shift_hessian



# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    wires = ins[0]
    weights = np.array(ins[1], requires_grad = True)
    output = compute_hessian(wires, weights)

    if isinstance(output,(tuple)):
        output = np.array(output).numpy().round(3)
        return str([elem.tolist() for elem in output])

    elif isinstance(output,(np.tensor)):

        return str(output.tolist())

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, atol=1e-2), "Your function does not calculate the Hessian correctly."


# These are the public test cases
test_cases = [
    ('[3,[0.1,0.2,0.1,0.2,0.7]]', '[[0.013, 0.0, 0.013, 0.006, 0.002], [0.0, -0.621, 0.077, 0.125, -0.604], [0.013, 0.077, -0.608, -0.628, -0.073], [0.006, 0.125, -0.628, 0.138, -0.044], [0.002, -0.604, -0.073, -0.044, -0.608]]'),
    ('[4,[0.78,0.23,0.54,-0.8,-0.3,0.0]]', '[[0.0, 0.0, 0.0, 0.0, 0.0, 0.128], [0.0, -0.582, 0.082, -0.14, 0.0, -0.343], [0.0, 0.082, -0.582, -0.359, 0.0, -0.057], [0.0, -0.14, -0.359, -0.582, 0.0, 0.204], [0.0, 0.0, 0.0, 0.0, 0.0, 0.393], [0.128, -0.343, -0.057, 0.204, 0.393, -0.582]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")